In [ ]:
import requests

1. Create a set of `visited` urls
2. Visit "Food" page
3. Grab the total number of food sub-pages from pagination element at bottom of page
4. Fill in the sub-page index to the general page $i$ url
5. On each page, open each product page and grab: 
    * Food category (`categories`)
    * Price
    * Weight
    * Ingredients
    * Serves X
    * Serving size
    * Calories per serving
    * Create a dictionary of `nutrient_amt`, `nutrient_dv`
    * `country_of_manufacture`
    * `country_of_origin`

## Scraping without running JS, `selenium`

In [ ]:
from bs4 import BeautifulSoup
import lxml


In [ ]:
# url_food_main = 'https://www.traderjoes.com/home/products/category/food-8' 
# # "Front page" for food
# # Get the max num of pages from "ul class=Pagination_pagination__list__1JUIg" (last item in that)

# html_food_main = requests.get(url_food_main).text
# soup_main = BeautifulSoup(html_food_main, 'html.parser')

# print(soup_main.prettify())

In [ ]:
# This is the url pattern for each page of food item results
subpage_num = 1 # Use .format() so that it we don't have to overwrite url_pattern each time
url_subpage_pattern = "https://www.traderjoes.com/home/products/category/food-8?filters={{\"page\"%3A{}}}"
url_subpage_pattern.format(subpage_num)


In [ ]:
# Grab the html from subpage 1, no headers or JS evaluation
html_subpage = requests.get(url_subpage_pattern.format(subpage_num)).text

In [ ]:
# Parse it
soup_nojs = BeautifulSoup(html_subpage)

In [ ]:
# Not finding elements we see in browser Inspect because the page gets populated by Javascript
soup_nojs.find_all('div', class_='aem-container aem-Grid aem-Grid--12 aem-Grid--default--12')

In [ ]:
# Using item SKU, we can get the item page directly, but how would we ever know the SKU ahead of time?
# This one is for Strawberry Doodle Cookies
fromsku = requests.get('https://traderjoes.com/home/products/pdp/081523').content
fromsku

In [ ]:
from bs4 import BeautifulSoup
import lxml
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


## GraphQL for grabbing Trader Joe's product info
An API querying language. Apparently, Trader Joe's keeps a product endpoint available without the need for an authentication token. I still don't know where someone would find out about this information (the fact that Trader Joe's has this, and the url exactly).

This way, we don't need to scrape or even render any JS. All we need to do now is understand the product schema. In particular, there seem to be tables for product 

In [129]:
url = "https://www.traderjoes.com/api/graphql"
# Where does ^ even come from!? Network tab under F12, AJAX requests idk

In [130]:
import requests

### Example query of 100 products and their prices

In [131]:
# Just mimicking the Haskell one

store_code = 706

query_string = """
query {
  products(search: "", pageSize: 100) {
    items {
      sku
      item_title
      retail_price
    }
    total_count
  }
}
"""

headers = {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9",
    "content-type": "application/json",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36",
}


query = {
    "operationName": "SearchProduct",
    "variables": {
        "storeCode": "706",
        "published": "1",
        "currentPage": 1,
        "pageSize": 100
    },
    "query": query_string
}

response = requests.post(url, json=query, headers=headers)

if response.status_code == 200:
    data = response.json()
    # Process the data as needed
    print(data)
else:
    print(f"Request failed with status code {response.status_code}")

{'data': {'products': {'items': [{'sku': '080626', 'item_title': 'Organic Baby Red Butter Lettuce & Baby Arugula', 'retail_price': '2.49'}, {'sku': '097210', 'item_title': 'Fair Trade Turbinado Raw Cane Sugar', 'retail_price': '3.49'}, {'sku': '082077', 'item_title': 'Mango Pineapple Pico de Gallo Salsa', 'retail_price': '3.99'}, {'sku': '080470', 'item_title': 'Strawberry & Watermelon Marshmallows', 'retail_price': '2.99'}, {'sku': '073162', 'item_title': 'Chocolate Croissants', 'retail_price': '3.49'}, {'sku': '082097', 'item_title': 'Sunshine Mountain Vineyard Pinot Noir 2023', 'retail_price': '14.99'}, {'sku': '079658', 'item_title': 'Strawberries & Cream Cloud Cake', 'retail_price': '6.49'}, {'sku': '080323', 'item_title': 'Italian Pistachio & Fior di Latte Gelato Style Frozen Dessert', 'retail_price': '4.99'}, {'sku': '080233', 'item_title': 'Cold Brew Coffee', 'retail_price': '3.29'}, {'sku': '080349', 'item_title': 'Hot Honey Fudge', 'retail_price': '2.99'}, {'sku': '079518', '

In [132]:
data['data']['products']['total_count']

25722

In [133]:
len(data['data']['products']['items'])

100

In [134]:
data['data']['products']['items']

[{'sku': '080626',
  'item_title': 'Organic Baby Red Butter Lettuce & Baby Arugula',
  'retail_price': '2.49'},
 {'sku': '097210',
  'item_title': 'Fair Trade Turbinado Raw Cane Sugar',
  'retail_price': '3.49'},
 {'sku': '082077',
  'item_title': 'Mango Pineapple Pico de Gallo Salsa',
  'retail_price': '3.99'},
 {'sku': '080470',
  'item_title': 'Strawberry & Watermelon Marshmallows',
  'retail_price': '2.99'},
 {'sku': '073162',
  'item_title': 'Chocolate Croissants',
  'retail_price': '3.49'},
 {'sku': '082097',
  'item_title': 'Sunshine Mountain Vineyard Pinot Noir 2023',
  'retail_price': '14.99'},
 {'sku': '079658',
  'item_title': 'Strawberries & Cream Cloud Cake',
  'retail_price': '6.49'},
 {'sku': '080323',
  'item_title': 'Italian Pistachio & Fior di Latte Gelato Style Frozen Dessert',
  'retail_price': '4.99'},
 {'sku': '080233', 'item_title': 'Cold Brew Coffee', 'retail_price': '3.29'},
 {'sku': '080349', 'item_title': 'Hot Honey Fudge', 'retail_price': '2.99'},
 {'sku': '

### "Introspecting" all defined queries in the API

These are all of the queries available to use. The `description` explains vaguely how they are used.

**NB: This is not telling us much about attributes that the queries return, only how to perform the query (how to search, filter, limit number of results).**

In [135]:
query_query_schema = """
{
  __type(name: "Query") {
    name
    kind
    description
    fields {
      name
      description
      args {
        name
        description
        type {
          name
          kind
          ofType {
            name
            kind
          }
        }
        defaultValue
      }
      type {
        name
        kind
        ofType {
          name
          kind
        }
      }
    }
  }
}
"""
response = requests.post(url, json={"query": query_query_schema}, headers=headers)
qschema = response.json()
qschema['data']['__type']['fields']


[{'name': 'availableStores',
  'description': 'Get a list of available store views and their config information.',
  'args': [{'name': 'useCurrentGroup',
    'description': 'Filter store views by the current store group.',
    'type': {'name': 'Boolean', 'kind': 'SCALAR', 'ofType': None},
    'defaultValue': None}],
  'type': {'name': None,
   'kind': 'LIST',
   'ofType': {'name': 'StoreConfig', 'kind': 'OBJECT'}}},
 {'name': 'cart',
  'description': 'Return information about the specified shopping cart.',
  'args': [{'name': 'cart_id',
    'description': 'The unique ID of the cart to query.',
    'type': {'name': None,
     'kind': 'NON_NULL',
     'ofType': {'name': 'String', 'kind': 'SCALAR'}},
    'defaultValue': None}],
  'type': {'name': 'Cart', 'kind': 'OBJECT', 'ofType': None}},
 {'name': 'categories',
  'description': 'Return a list of categories that match the specified filter.',
  'args': [{'name': 'filters',
    'description': 'Identifies which Category filter inputs to sea

In [ ]:
# {'name': 'products',
#   'description': 'Search for products that match the criteria specified in the `search` and `filter` attributes.',
#   'args': [
#    {'name': 'search',
#     'description': 'One or more keywords to use in a full-text search.',
#     'type': {'name': 'String', 'kind': 'SCALAR', 'ofType': None},
#     'defaultValue': None},
#    {'name': 'filter',
#     'description': 'The product attributes to search for and return.',
#     'type': {'name': 'ProductAttributeFilterInput',
#      'kind': 'INPUT_OBJECT',
#      'ofType': None},
#     'defaultValue': None},
#    {'name': 'pageSize',
#     'description': 'The maximum number of results to return at once. The default value is 20.',
#     'type': {'name': 'Int', 'kind': 'SCALAR', 'ofType': None},
#     'defaultValue': '20'},
#    {'name': 'currentPage',
#     'description': 'The page of results to return. The default value is 1.',
#     'type': {'name': 'Int', 'kind': 'SCALAR', 'ofType': None},
#     'defaultValue': '1'},
#    {'name': 'sort',
#     'description': 'Specifies which attributes to sort on, and whether to return the results in ascending or descending order.',
#     'type': {'name': 'ProductAttributeSortInput',
#      'kind': 'INPUT_OBJECT',
#      'ofType': None},
#     'defaultValue': None}
#    ],
#   'type': {'name': 'Products', 'kind': 'OBJECT', 'ofType': None}}

This actually gets the fields in a `Products` object.

In [136]:
query_product_fields = """
{
  __type(name: "Products") {
    name
    kind
    fields {
      name
      description
      type {
        name
        kind
        ofType {
          name
          kind
        }
      }
    }
  }
}
"""
product_fields_response = requests.post(url, json={"query": query_product_fields}, headers=headers)

product_fields = product_fields_response.json()
# product_fields

In [137]:
product_fields['data']['__type']['fields']

[{'name': 'aggregations',
  'description': 'A bucket that contains the attribute code and label for each filterable option.',
  'type': {'name': None,
   'kind': 'LIST',
   'ofType': {'name': 'Aggregation', 'kind': 'OBJECT'}}},
 {'name': 'items',
  'description': 'An array of products that match the specified search criteria.',
  'type': {'name': None,
   'kind': 'LIST',
   'ofType': {'name': 'ProductInterface', 'kind': 'INTERFACE'}}},
 {'name': 'page_info',
  'description': 'An object that includes the page_info and currentPage values specified in the query.',
  'type': {'name': 'SearchResultPageInfo', 'kind': 'OBJECT', 'ofType': None}},
 {'name': 'sort_fields',
  'description': 'An object that includes the default sort field and all available sort fields.',
  'type': {'name': 'SortFields', 'kind': 'OBJECT', 'ofType': None}},
 {'name': 'suggestions',
  'description': 'An array of search suggestions for case when search query have no results.',
  'type': {'name': None,
   'kind': 'LIST

So apparently, `ProductInterface` is the actual class we are concerned with, as `Product` only contains an `items` attribute of many products that matched the given query.

In [138]:
query_productinterface_fields = """
{
  __type(name: "ProductInterface") {
    name
    kind
    fields {
      name
      description
      type {
        name
        kind
        ofType {
          name
          kind
        }
      }
    }
  }
}
"""
productinterface_fields_response = requests.post(url, json={"query": query_productinterface_fields}, headers=headers)

productinterface_fields = productinterface_fields_response.json()

In [139]:
productinterface_fields['data']['__type']['fields']

[{'name': 'all_context_images',
  'description': 'Array of all context images',
  'type': {'name': None,
   'kind': 'LIST',
   'ofType': {'name': 'StoreSpecificImages', 'kind': 'OBJECT'}}},
 {'name': 'all_other_images',
  'description': 'Array of all other image attributes info',
  'type': {'name': None,
   'kind': 'LIST',
   'ofType': {'name': 'StoreSpecificImages', 'kind': 'OBJECT'}}},
 {'name': 'all_primary_images',
  'description': 'Array of all primary images',
  'type': {'name': None,
   'kind': 'LIST',
   'ofType': {'name': 'StoreSpecificImages', 'kind': 'OBJECT'}}},
 {'name': 'allergens',
  'description': "Product's Allergens",
  'type': {'name': None,
   'kind': 'LIST',
   'ofType': {'name': 'IngredientsAttribute', 'kind': 'OBJECT'}}},
 {'name': 'availability',
  'description': 'Availability on the specified store',
  'type': {'name': 'String', 'kind': 'SCALAR', 'ofType': None}},
 {'name': 'canonical_url',
  'description': "The relative canonical URL. This value is returned on

OK, we are getting somewhere. We now know that `Products` contains a list `items` of `ProductInterface`s, which then have their own `nutrition` attribute of type `NutritionAttribute`.

I was confused by the naming scheme, but it is clear now that `NutritionAttribute` is not an attribute of a particular nutrition object, but the attribute containing nutrition information of a parent `ProductInterface`.

In [ ]:
query_nutrition_fields = """
{
  __type(name: "NutritionAttribute") {
    name
    kind
    fields {
      name
      description
      type {
        name
        kind
        ofType {
          name
          kind
        }
      }
    }
  }
}
"""
query_nutrition_fields_response = requests.post(url, json={"query": query_nutrition_fields}, headers=headers)

nutrition_fields = query_nutrition_fields_response.json()

In [ ]:
[(field['name']) for field in nutrition_fields['data']['__type']['fields']]
# nutrition_fields['data']['__type']['fields']

To summarize all of the objects and their fields that we desire:

```
Products
|
L Attributes include:
    L sku
    L item_title
    L retail_price
    L items
        L nutrition
            L calories_per_serving
            L details
            L serving_size
            L servings_per_container
L

```

In [ ]:
# {'name': 'route',
#   'description': 'Return the full details for a specified product, category, or CMS page.',
#   'args': [{'name': 'url',
#     'description': 'A `url_key` appended by the `url_suffix, if one exists.',
#     'type': {'name': None,
#      'kind': 'NON_NULL',
#      'ofType': {'name': 'String', 'kind': 'SCALAR'}},
#     'defaultValue': None}],
#   'type': {'name': 'RoutableInterface', 'kind': 'INTERFACE', 'ofType': None}}


In [ ]:
# query_string = """
# query {
#   route(url: "https://www.traderjoes.com/api/graphql") {
#     __typename
#     ... on Product {
#       sku
#     }
#   }
# }
# """

# query = {
#     "operationName": "SearchRoute",
#     "variables": {
#         "storeCode": "706",
#         "published": "1",
#         "currentPage": 1,
#         "pageSize": 100
#     },
#     "query": query_string
# }

# response = requests.post(url, json=query, headers=headers)

# # data = response.json()
# # data
# response

# How to use this `routes` query?

Let's grab the full API schema and see if we can find which fields to query for our purposes.

In [ ]:
query_schema = """
query {
  __schema {
    types {
      name
      kind
      description
    }
  }
}
"""

response = requests.post(url, json={"query": query_schema}, headers=headers)
schema = response.json()
# print(schema)

In [ ]:
schema['data']['__schema']['types'] # ['data']['__type'].keys()

In [ ]:

query_schema_nutrition = """
{
  __type(name: "NutritionDetails") {
    name
    kind
    description
    fields {
      name
      type {
        kind
        name
        ofType {
          kind
          name
        }
      }
    }
  }
}"""

response = requests.post(url, json={"query": query_schema_nutrition}, headers=headers)
nutrition = response.json()
print(nutrition)

What is the relationship between `NutritionDetails` and `NutritionAttribute`?

In [ ]:
nutrition['data']['__type']['fields']

In [ ]:
query_schema_all = """
{
  __schema {
    queryType {
      name
    }
    mutationType {
      name
    }
    subscriptionType {
      name
    }
    types {
      name
      kind
      fields {
        name
        args {
          name
          type {
            name
            kind
          }
          defaultValue
        }
        type {
          name
          kind
        }
      }
      inputFields {
        name
        type {
          name
          kind
        }
      }
      interfaces {
        name
      }
      enumValues {
        name
      }
      possibleTypes {
        name
      }
    }
  }
}
"""
response = requests.post(url, json={"query": query_schema_all}, headers=headers)
schema_all = response.json()
# print(schema_all)

In [ ]:
schema_all['data']['__schema']['types']

### Finding fields in various attributes

This is how we get all available fields in an Object `NutritionAttribute`.

In [ ]:
# NutritionAttribute
# IngredientsAttribute
query_nutschema = """
query {
  __type(name: "ComplexTextValue") {
    name
    fields {
      name
      type {
        name
        kind
      }
    }
  }
}
"""

out = requests.post(url, json={"query": query_nutschema}, headers=headers)
out = out.json()
# print(out)
out['data']['__type']

In [ ]:
query_prodschema = """
query {
  __type(name: "Products") {
    name
    fields {
      name
      type {
        name
        kind
      }
    }
  }
}
"""

out = requests.post(url, json={"query": query_prodschema}, headers=headers)
out = out.json()
print(out)

In [ ]:
out

In [ ]:
query_types_schema = """
query {
  __schema {
    types {
      name
    }
  }
}
"""

out = requests.post(url, json={"query": query_types_schema}, headers=headers)
out = out.json()
out

### Test out querying one product

In [ ]:
query_one_product = """
{
  products(
    storeCode: "706",
    search: "",
    filter: {},
    pageSize: 3,
    currentPage: 0
  ) {
    items {
      sku
      name
      availability
    }
  }
}
"""

response = requests.post(url, json={"query": query_one_product}, headers=headers)

# Check if the response is successful
if response.status_code == 200:
    one_product = response.json()
else:
    # Handle potential errors
    raise Exception(f"Query failed with status code {response.status_code}: {response.text}")

print(one_product)


In [ ]:
one_product

### Only querying items at Hyde Park location (and available?)

In [ ]:
import requests
import json

def items_by_store(store_code, page):
    url = "https://www.traderjoes.com/api/graphql"
    headers = {
        "accept": "*/*",
        "accept-language": "en-US,en;q=0.9",
        "cache-control": "no-cache",
        "content-type": "application/json",
        "pragma": "no-cache",
        "accept-encoding": "gzip, deflate, br",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 \
                       (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36"
    }

    query = f"""
    {{
      products(
        search: "",
        filter: {{}},
        currentPage: {page},
        pageSize: 100
      ) {{
        items {{
          sku
          name
          availability
        }}
        total_count
      }}
    }}
    """

    # Construct the payload
    payload = {
        "operationName": "SearchProduct",
        "query": query
    }

    # Send the request
    response = requests.post(url, json=payload, headers=headers)

    # Check if the response is successful
    if response.status_code == 200:
        try:
            data = response.json()
            return data.get('data', {}).get('products', {}).get('items', [])
        except json.JSONDecodeError:
            raise Exception("Failed to parse response JSON")
    else:
        raise Exception(f"Query failed with status code {response.status_code}: {response.text}")

# Example usage
items = items_by_store("706", 1)
for item in items:
    print(item)

In [ ]:
items[0]['availability']

In [ ]:
len(items)

In [ ]:
[item for item in items if item['availability']=='0']

### Actually query all fields from all products (including nutrition)

Now, query the nutritional information.

uid vs sku? Both unique identifiers.

In [ ]:
# Get total number of products
query_number_products = """
{
  products(
    search: "",
    filter: {},
    pageSize: 100,
    currentPage: 0
  ) 
  {
    total_count
    page_info {
      current_page
      page_size
      total_pages
    }
  }
}
"""
number_products = requests.post(url, json={"query": query_number_products}, headers=headers)
number_products = number_products.json()
# print(number_products)


In [ ]:
number_products['data']

Let's query 256 pages of 100 products.

In [140]:
page_info = number_products['data']['products']['page_info']
page_size = page_info['page_size']
num_pages = page_info['total_pages']

query_product_page = """
{{
  products(
    search: "",
    filter: {{}},
    pageSize: {},
    currentPage: {}
  )
  {{
    items {{
      sku
      name
      nutrition {{
        calories_per_serving
        details {{
            amount
            display_seq
            nutritional_item
            percent_dv
        }}
        display_sequence
        panel_id
        panel_title
        serving_size
        servings_per_container
      }}
      ingredients {{
        display_sequence
        ingredient
      }}
      item_description
      popularity
      price {{
        regularPrice {{
          amount {{
            value
            currency
          }}
        }}
      }}
      country_of_manufacture
      country_of_origin
      description {{
        html
      }}
    }}
    total_count
    page_info {{
      current_page
      page_size
      total_pages
    }}
  }}
}}
"""


In [ ]:
num_pages

In [ ]:
all_items = []

In [ ]:
# # Last run time: 3m 42.6s!
# # Page 0 is identical to page 1!!
# for page_idx in range(1, num_pages+1):
#     query_page_i = {"query": query_product_page.format(page_size, page_idx)}
#     product_page = requests.post(url, json=query_page_i, headers=headers)
#     product_page = product_page.json()
#     all_items = all_items + product_page['data']['products']['items']


In [128]:
len(all_items)

0

In [ ]:
skus = [item['sku'] for item in all_items]

In [ ]:
len(skus)

In [ ]:
len(set(skus))

In [ ]:
25564

In [ ]:
len(set(skus[100:])) == len(set(skus))
# Ah-ha! Yes, page 0 is identical to page 1

In [ ]:
query_page_256 = {"query": query_product_page.format(page_size, 256)}
product_page_256 = requests.post(url, json=query_page_256, headers=headers)
product_page_256 = product_page_256.json()
all_items_fixed = all_items[100:]

all_items_fixed = all_items_fixed + product_page_256['data']['products']['items']

In [ ]:
skus_fixed = [item['sku'] for item in all_items_fixed]
len(set(skus_fixed))

### Query all items from Hyde Park location

In [144]:
query_product_page = """
{{
  products(
    search: "",
    filter: {{}},
    pageSize: {},
    currentPage: {}
  )
  {{
    items {{
      sku
      name
      availability
      stock_status
      only_x_left_in_stock
      nutrition {{
        calories_per_serving
        details {{
            amount
            display_seq
            nutritional_item
            percent_dv
        }}
        display_sequence
        panel_id
        panel_title
        serving_size
        servings_per_container
      }}
      ingredients {{
        display_sequence
        ingredient
      }}
      item_description
      popularity
      price {{
        regularPrice {{
          amount {{
            value
            currency
          }}
        }}
      }}
      country_of_manufacture
      country_of_origin
      description {{
        html
      }}
    }}
    total_count
    page_info {{
      current_page
      page_size
      total_pages
    }}
  }}
}}
"""

In [145]:
# Just mimicking the Haskell one

store_code = 706

headers = {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9",
    "content-type": "application/json",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36",
}

page_idx=1
query_product_page_filled = query_product_page.format(page_size, page_idx)
query = {
    "operationName": "SearchProduct",
    "variables": {
        "storeCode": "706",
        "published": "1",
        "currentPage": 1,
        "pageSize": 100
    },
    "query": query_product_page_filled
}

response = requests.post(url, json=query, headers=headers)

if response.status_code == 200:
    data = response.json()
    # Process the data as needed
    print(data)
else:
    print(f"Request failed with status code {response.status_code}")

{'data': {'products': {'items': [{'sku': '080626', 'name': 'R-SALAD BABY RED BUTTER & ARUGULA ORG', 'availability': '1', 'stock_status': 'OUT_OF_STOCK', 'only_x_left_in_stock': None, 'nutrition': [{'calories_per_serving': '15 ', 'details': [{'amount': '0 g', 'display_seq': 1, 'nutritional_item': 'Total Fat', 'percent_dv': '0'}, {'amount': '0 g', 'display_seq': 2, 'nutritional_item': 'Saturated Fat', 'percent_dv': '0'}, {'amount': '0 g', 'display_seq': 3, 'nutritional_item': 'Trans Fat', 'percent_dv': None}, {'amount': '0 mg', 'display_seq': 4, 'nutritional_item': 'Cholesterol', 'percent_dv': '0'}, {'amount': '10 mg', 'display_seq': 5, 'nutritional_item': 'Sodium', 'percent_dv': '0'}, {'amount': '2 g', 'display_seq': 6, 'nutritional_item': 'Total Carbohydrate', 'percent_dv': '.01'}, {'amount': '1 g', 'display_seq': 7, 'nutritional_item': 'Dietary Fiber', 'percent_dv': '.04'}, {'amount': '1 g', 'display_seq': 8, 'nutritional_item': 'Total Sugars', 'percent_dv': None}, {'amount': '0 g Add

In [184]:
num_pages

258

In [150]:
page_size

100

In [ ]:
# all_items_hp = []
# # Last run time: 5m 47.0s!
# # Page 0 is identical to page 1!!
# for page_idx in range(1, num_pages+1):
    
#     query_product_page_filled = query_product_page.format(page_size, page_idx)
#     query_page_i = {
#         "operationName": "SearchProduct",
#         "variables": {
#             "storeCode": "706",
#             "published": "1",
#             "currentPage": 1,
#             "pageSize": 100
#         },
#         "query": query_product_page_filled
#     }
#     # query_page_i = {"query": query_product_page.format(page_size, page_idx)}
#     product_page = requests.post(url, json=query_page_i, headers=headers)
#     product_page = product_page.json()
#     all_items_hp = all_items_hp + product_page['data']['products']['items']


In [ ]:
# len(all_items_hp) # 25722 on 06/29/2025

25722

## Converting json output to `polars`

### Hyde Park list

In [156]:
skus = [item['sku'] for item in all_items_hp]

In [157]:
len(skus)

25722

In [158]:
len(set(skus))

23150

In [160]:
num_pages

258

In [165]:
from collections import Counter

In [169]:
skus = [i['sku'] for i in all_items_hp]
skus_ct = Counter(skus)

In [182]:
# skus_ct

In [180]:
sum([1 if i==2 else 0 for i in skus_ct.values()])

2572

In [183]:
[i for i in all_items_hp if i['sku']=='007363']

[{'sku': '007363',
  'name': 'JAGERMEISTER LIQUEUR 750ML',
  'availability': '1',
  'stock_status': 'OUT_OF_STOCK',
  'only_x_left_in_stock': None,
  'nutrition': None,
  'ingredients': None,
  'item_description': None,
  'popularity': '0',
  'price': {'regularPrice': {'amount': {'value': 18.99, 'currency': 'USD'}}},
  'country_of_manufacture': None,
  'country_of_origin': None,
  'description': {'html': ''}},
 {'sku': '007363',
  'name': 'JAGERMEISTER LIQUEUR 750ML',
  'availability': '1',
  'stock_status': 'OUT_OF_STOCK',
  'only_x_left_in_stock': None,
  'nutrition': None,
  'ingredients': None,
  'item_description': None,
  'popularity': '0',
  'price': {'regularPrice': {'amount': {'value': 18.99, 'currency': 'USD'}}},
  'country_of_manufacture': None,
  'country_of_origin': None,
  'description': {'html': ''}}]

In [168]:
len(all_items_hp)

25722

Uh-oh, remove duplicates and sponges.

Oh, sku 10032021 "POP UP SPONGES" for some reason contains information for pizza crusts? The problematic `"item_description": "Broccoli & Kale Pizza Crust description"`.

In [191]:
all_items_hp_fixed = [item for item in all_items_hp if item['sku']!='10032021']

In [192]:
len(all_items_hp_fixed)

25721

Save Hyde Park json.

In [ ]:
# import json
# all_items_hp_fixed_raw_path = "data/all_items_hp_fixed_raw.json"
# with open(all_items_hp_fixed_raw_path, "w") as f:
#        json.dump(all_items_hp_fixed, f, indent=4)



In [196]:
import polars as pl

In [ ]:
df_hp = pl.json_normalize(all_items_hp_fixed)
df_hp.head(5)

sku,name,availability,stock_status,only_x_left_in_stock,nutrition,ingredients,item_description,popularity,country_of_manufacture,country_of_origin,price.regularPrice.amount.value,price.regularPrice.amount.currency,description.html
str,str,str,str,null,list[struct[7]],list[struct[2]],null,str,null,str,f64,str,str
"""080626""","""R-SALAD BABY RED BUTTER & ARUG…","""1""","""OUT_OF_STOCK""",null,"[{""15 "",[{""0 g"",1,""Total Fat"",""0""}, {""0 g"",2,""Saturated Fat"",""0""}, … {""250 mg"",14,""Potassium"","".06""}],0,1,""per serving"",""1/2 package (85g)"",""Serves 2""}, {""30 "",[{""0.5 g"",1,""Total Fat"","".01""}, {""0 g"",2,""Saturated Fat"",""0""}, … {""490 mg"",14,""Potassium"","".1""}],1,2,""per container"",""1/2 package (85g)"",""Serves 2""}]","[{1,""ORGANIC BABY RED BUTTER LETTUCE""}, {2,""ORGANIC BABY ARUGULA""}]",null,"""107""",null,"""Product of USA""",2.49,"""USD""",""""""
"""097210""","""TURBINADO RAW CANE SUGAR""","""1""","""OUT_OF_STOCK""",null,"[{""30"",[{""0 g"",1,""Total Fat"",""0""}, {""0 g"",2,""Saturated Fat"",""0""}, … {""0 mg"",14,""Potassium"",""0""}],0,1,"""",""2 tsp (8 g)"",""Serves 85""}]","[{1,""TURBINADO RAW CANE SUGAR""}]",null,"""95""",null,"""Product of Malawi Product of""",3.49,"""USD""",""""""
"""082077""","""SALSA MANGO PINEAPPLE PICO DE …","""1""","""OUT_OF_STOCK""",null,"[{""10 "",[{""0 g"",1,""Total Fat"",""0""}, {""0 g"",2,""Saturated Fat"",""0""}, … {""50 mg"",14,""Potassium"","".02""}],0,1,null,""2 Tbsp. (30g)"",""Serves about 11""}]","[{1,""TOMATO""}, {2,""YELLOW ONION""}, … {8,""SALT""}]",null,"""236""",null,"""Made in United States""",3.99,"""USD""",""""""
"""080470""","""MARSHMALLOWS STRAWBERRY & WATE…","""1""","""OUT_OF_STOCK""",null,"[{""100 "",[{""0 g"",1,""Total Fat"",""0""}, {""0 g"",2,""Saturated Fat"",""0""}, … {""0 mg"",14,""Potassium"",""0""}],0,1,null,""6 pieces (30g)"",""Serves about 8""}]","[{1,""GLUCOSE-FRUCTOSE SYRUP""}, {2,""SUGAR""}, … {9,""WATERMELON POWDER (MALTODEXTRIN, WATERMELON JUICE CONCENTRATE, CITRIC ACID [ACIDIFIER]).""}]",null,"""371""",null,"""Product of Spain""",2.99,"""USD""",""""""
"""073162""","""CROISSANT CHOCOLATE""","""1""","""OUT_OF_STOCK""",null,"[{""350 "",[{""20 g"",1,""Total Fat"","".26""}, {""12 g"",2,""Saturated Fat"","".6""}, … {""140 mg"",14,""Potassium"","".02""}],0,1,""Per serving"",""1 croissant (83g)"",""Serves 2""}, {""710 "",[{""40 g"",1,""Total Fat"","".51""}, {""23 g"",2,""Saturated Fat"",""1.15""}, … {""280 mg"",14,""Potassium"","".06""}],1,2,""Per container"",""1 croissant (83g)"",""Serves 2""}]","[{1,""UNBLEACHED WHEAT FLOUR (WHEAT FLOUR, ENZYME)""}, {2,""WATER""}, … {12,""ENZYMES.""}]",null,"""158""",null,"""Product of USA""",3.49,"""USD""",""""""


Beyond just dumping each json object from `all_items_fixed` into `pl.json_normalize()`, we also need to further handle the `nutrition` and `ingredients` dictionaries.

In [ ]:
# df_all_items_raw.write_csv('data/all_items_raw.csv')
# # Nested columns 2 and 3 do not work with CSV

In [ ]:
# import json
# all_items_fixed_raw_path = "data/all_items_fixed_raw.json"
# with open(all_items_fixed_raw_path, "w") as f:
#        json.dump(all_items_fixed, f, indent=4)



Every description is empty html???

Let's try and isolate only food items.

In [199]:
nuts = [item['nutrition'] for item in all_items_hp_fixed]

In [ ]:
# Get only items with non-empty nutrition information
# This will be criterion for what counts as "food"

In [203]:
# nut_lens = [len(nut) for nut in nuts]

#### Getting only the values we need from the json, making it relational

In [ ]:
import json
import polars as pl

In [204]:
with open('data/all_items_hp_fixed_raw.json', 'r') as f:
    tjhp_raw = json.load(f)

Recall that the `item_description` field is `null` for just about every entry except one where it is instead the unrelated `Broccoli & Kale Pizza Crust description`.

In [ ]:
# tj_raw = [item for item in tj_raw if item['item_description'] != 'Broccoli & Kale Pizza Crust description']

In [ ]:
tjhp = pl.DataFrame(tjhp_raw)

Need to:

* Remove rows with null `nutrition`
* Remove useless fields (`item_description`, `description`)
* Drop duplicate rows
* Turn nested fields into 
    * lists (ingredients)
    * individual and possibly sparse columns (fiber, protein, calories)

In [206]:
tjhp = tjhp.drop(['item_description', 'description'])

In [207]:
tjhp.shape

(25721, 11)

In [208]:
tjhp = tjhp.drop_nulls(subset=['nutrition', 'price'])
tjhp.shape

(4783, 11)

In [ ]:
# Drop dupes
tjhp = tjhp.unique()

In [ ]:
# fix price so it actually is a float of US dollars
temp = tjhp.with_columns(
    pl.col("price").struct.field("regularPrice").struct.field("amount").struct.field("currency").alias("nested_field")
)

print(temp)

shape: (4_588, 12)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ sku    ┆ name       ┆ availabili ┆ stock_sta ┆ … ┆ price     ┆ country_o ┆ country_o ┆ nested_fi │
│ ---    ┆ ---        ┆ ty         ┆ tus       ┆   ┆ ---       ┆ f_manufac ┆ f_origin  ┆ eld       │
│ str    ┆ str        ┆ ---        ┆ ---       ┆   ┆ struct[1] ┆ ture      ┆ ---       ┆ ---       │
│        ┆            ┆ str        ┆ str       ┆   ┆           ┆ ---       ┆ str       ┆ str       │
│        ┆            ┆            ┆           ┆   ┆           ┆ null      ┆           ┆           │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 075259 ┆ MEAL       ┆ 1          ┆ OUT_OF_ST ┆ … ┆ {{{3.99," ┆ null      ┆ Product   ┆ USD       │
│        ┆ CASHEW     ┆            ┆ OCK       ┆   ┆ USD"}}}   ┆           ┆ of Sri    ┆           │
│        ┆ CURRY &    ┆            ┆           ┆   ┆           ┆        

In [216]:
temp['nested_field'].value_counts()

nested_field,count
str,u32
"""USD""",4588


OK, they are all in USD, proceed.

In [217]:
tjhp = tjhp.with_columns(
    pl.col("price").struct.field("regularPrice").struct.field("amount").struct.field("value").alias("price_usd")
)


Onto the same nested json but for calories...

Wow, these nutrition items are formatted horrendously. The key is not a key whatsoever. Should go back and fix this in the original GraphQL query under `nutrition`.

In [218]:
tjhp['nutrition'][0]

""
struct[7]
"{""610 "",[{""38 g"",1,""Total Fat"","".49""}, {""25 g"",2,""Saturated Fat"",""1.25""}, … {""530 mg"",14,""Potassium"","".1""}],0,1,""Per 1 cup (250g)"",""1 package (375g)"",""Serves 1""}"
"{""910 "",[{""56 g"",1,""Total Fat"","".72""}, {""37 g"",2,""Saturated Fat"",""1.85""}, … {""790 mg"",14,""Potassium"","".15""}],1,2,""Per container"",""1 package (375g)"",""Serves 1""}"


Oh, maybe not.

In [219]:
tjhp['nutrition'][0][0]

{'calories_per_serving': '610 ',
 'details': [{'amount': '38 g',
   'display_seq': 1,
   'nutritional_item': 'Total Fat',
   'percent_dv': '.49'},
  {'amount': '25 g',
   'display_seq': 2,
   'nutritional_item': 'Saturated Fat',
   'percent_dv': '1.25'},
  {'amount': '0 g',
   'display_seq': 3,
   'nutritional_item': 'Trans Fat',
   'percent_dv': ''},
  {'amount': '0 mg',
   'display_seq': 4,
   'nutritional_item': 'Cholesterol',
   'percent_dv': '0'},
  {'amount': '500 mg',
   'display_seq': 5,
   'nutritional_item': 'Sodium',
   'percent_dv': '.22'},
  {'amount': '58 g',
   'display_seq': 6,
   'nutritional_item': 'Total Carbohydrate',
   'percent_dv': '.21'},
  {'amount': '4 g',
   'display_seq': 7,
   'nutritional_item': 'Dietary Fiber',
   'percent_dv': '.14'},
  {'amount': '4 g',
   'display_seq': 8,
   'nutritional_item': 'Total Sugars',
   'percent_dv': ''},
  {'amount': '0 g Added Sugars',
   'display_seq': 9,
   'nutritional_item': 'Includes',
   'percent_dv': '0'},
  {'amoun

In [220]:
tjhp['nutrition'][0][0]['calories_per_serving']

'610 '

In [221]:
tjhp = tjhp.with_columns(
    # tjhp['nutrition']
    pl.col("nutrition").map_elements(lambda x: x[0]['calories_per_serving'] if len(x)>0 else None, return_dtype=str).alias("calories_per_serving")
)
tjhp.head(3)

sku,name,availability,stock_status,only_x_left_in_stock,nutrition,ingredients,popularity,price,country_of_manufacture,country_of_origin,price_usd,calories_per_serving
str,str,str,str,null,list[struct[7]],list[struct[2]],str,struct[1],null,str,f64,str
"""075259""","""MEAL CASHEW CURRY & RICE""","""1""","""OUT_OF_STOCK""",null,"[{""610 "",[{""38 g"",1,""Total Fat"","".49""}, {""25 g"",2,""Saturated Fat"",""1.25""}, … {""530 mg"",14,""Potassium"","".1""}],0,1,""Per 1 cup (250g)"",""1 package (375g)"",""Serves 1""}, {""910 "",[{""56 g"",1,""Total Fat"","".72""}, {""37 g"",2,""Saturated Fat"",""1.85""}, … {""790 mg"",14,""Potassium"","".15""}],1,2,""Per container"",""1 package (375g)"",""Serves 1""}]","[{1,""ORGANIC COOKED SEASONED RICE PACKET (ORGANIC RICE, WATER, ORGANIC PARSLEY, ORGANIC CURRY LEAVES, ORGANIC NATURAL FLAVOR, ORGANIC COCONUT OIL, ORGANIC BLACK PEPPER, ORGANIC CUMIN SEED, ORGANIC CINNAMON, ORGANIC TURMERIC POWDER, ORGANIC CARDAMOM, SEA SALT)""}, {2,""ORGANIC SAUCE PACKET (ORGANIC COCONUT MILK, WATER, ORGANIC ONIONS, ORGANIC GARLIC, ORGANIC GINGER, ORGANIC CORIANDER, ORGANIC CUMIN SEED, ORGANIC CURRY LEAVES, ORGANIC TURMERIC POWDER, SEA SALT, ORGANIC CINNAMON, ORGANIC BLACK PEPPER, ORGANIC CARDAMOM)""}, {3,""ORGANIC CASHEW PACKET (ORGANIC CASHEWS)""}]","""49""","{{{3.99,""USD""}}}",null,"""Product of Sri Lanka""",3.99,"""610 """
"""076838""","""HOL UNEXPECTED CHEDDAR SPREAD-…","""1""","""OUT_OF_STOCK""",null,"[{""90 "",[{""6 g"",1,""Total Fat"","".08""}, {""4.0 g"",2,""Saturated Fat"","".2""}, … {""180 mg"",14,""Potassium"","".04""}],0,1,"""",""2 Tbsp. (28g)"",""Serves about 9""}]","[{1,""CHEDDAR CHEESE (PASTEURIZED MILK, CHEESE CULTURES, SALT, MICROBIAL ENZYME)""}, {2,""WATER""}, … {11,""CINNAMON.""}]","""10""","{{{4.99,""USD""}}}",null,"""Product of United States""",4.99,"""90 """
"""066838""","""SALMON HOT SMOKED 5 OZ""","""1""","""OUT_OF_STOCK""",null,"[{""110 "",[{""7 g"",1,""Total Fat"","".09""}, {""1.5 g"",2,""Saturated Fat"","".08""}, … {""210 mg"",14,""Potassium"","".04""}],0,1,""Per Serving"",""2oz (56g)"",""Serves about 2.5""}, {""290 "",[{""17 g"",1,""Total Fat"","".22""}, {""3.5 g"",2,""Saturated Fat"","".18""}, … {""530 mg"",14,""Potassium"","".1""}],1,2,""Per Container"",""2oz (56g)"",""Serves about 2.5""}]","[{1,""FARM RAISED ATLANTIC SALMON [COLOR ADDED]""}, {2,""SEA SALT""}, {3,""HARDWOOD SMOKE.""}]","""1""","{{{5.99,""USD""}}}",null,"""Raised in Scotland Processed i…",5.99,"""110 """


These things definitely do not actually have 0 calories. Drop them.

In [222]:
tjhp.filter((pl.col("calories_per_serving") == ""))

sku,name,availability,stock_status,only_x_left_in_stock,nutrition,ingredients,popularity,price,country_of_manufacture,country_of_origin,price_usd,calories_per_serving
str,str,str,str,null,list[struct[7]],list[struct[2]],str,struct[1],null,str,f64,str
"""063197""","""GRATIN BROCCOLI & CAULIFLOWER""","""1""","""IN_STOCK""",null,"[{"""",[{""7 g"",1,""Total Fat"","".09""}, {""4.0 g"",2,""Saturated Fat"","".2""}, … {""260 mg"",14,""Potassium"","".06""}],0,1,"""",""1 cup (150g)"",""about 4""}]","[{1,""BROCCOLI""}, {2,""CAULIFLOWER""}, … {11,""BLACK PEPPER.""}]","""2""","{{{4.49,""USD""}}}",null,"""Product of Italy""",4.49,""""""
"""066054""","""CHEESECAKE CONES""","""1""","""IN_STOCK""",null,"[{"""",[{""13 g"",1,""Total Fat"","".17""}, {""5 g"",2,""Saturated Fat"","".25""}, … {""120 mg"",14,""Potassium"","".02""}],0,1,"""",""1 cone (50g)"",""4""}]","[{1,""CONE (SUGAR, ALMONDS""}, {2,""GLUCOSE SYRUP""}, … {13,""SUGAR).""}]","""147""","{{{4.99,""USD""}}}",null,"""Product of Italy""",4.99,""""""
"""072630""","""JOE'S CARVERY JERKY BITES""","""1""","""OUT_OF_STOCK""",null,"[{"""",[{"""",1,""CRUDE PROTEIN (MIN)"","".22""}, {"""",2,""CRUDE FAT (MIN)"","".1""}, … {"""",4,""MOISTURE (MAX)"","".22""}],0,1,"""",""1 treat"",""CALORIE CONTENT (CALCULATED) ME: 3500 kcal/kg; 25 kcal/treat""}]","[{1,""TURKEY""}, {2,""CHICKEN""}, … {15,""ROSEMARY EXTRACT.""}]","""19""","{{{3.49,""USD""}}}",null,"""Made in USA""",3.49,""""""
"""076648""","""HOL PB DECO DOG COOKIES""","""1""","""OUT_OF_STOCK""",null,"[{"""",[{"""",1,""CRUDE PROTEIN (MIN)"","".09""}, {"""",2,""CRUDE FAT (MIN)"","".05""}, … {"""",4,""MOISTURE (MAX)"","".12""}],0,1,"""",""3400 kcal/kg"",""45 kcal/treat""}]","[{1,""WHEAT FLOUR""}, {2,""GLUCOSE""}, … {17,""CALCIUM CARBONATE.""}]","""17""","{{{3.99,""USD""}}}",null,"""Product of Vietnam""",3.99,""""""
"""069471""","""HOL CREAMED GREENS""","""1""","""IN_STOCK""",null,"[{"""",[{""7 g"",1,""Total Fat"","".09""}, {""2.5 g"",2,""Saturated Fat"","".13""}, … {""190 mg"",14,""Potassium"","".04""}],0,1,"""",""1 cup (124g)"",""Serves about 4""}]","[{1,""MILK (MILK, VITAMIN D3), BRUSSELS SPROUTS, KALE""}, {2,""ONION""}, … {11,""DRIED ROSEMARY.""}]","""10""","{{{5.99,""USD""}}}",null,"""Manufactured in USA""",5.99,""""""
"""070148""","""MILK ULTRA - FILTERED REDUCED …","""1""","""IN_STOCK""",null,"[{"""",[{""5g"",1,""Total Fat"","".06""}, {""3g"",2,""Saturated Fat"","".15""}, … {""200mcg"",15,""Vitamin A"","".2""}],0,1,"""",""1 cup (240mL)"",""Serves about 7""}]","[{1,""ULTRA-FILTERED REDUCED FAT MILK""}, {2,""WATER""}, … {5,""VITAMIN D3""}]","""84""","{{{3.99,""USD""}}}",null,"""Product of United States""",3.99,""""""


In [223]:
tjhp = tjhp.filter((pl.col("calories_per_serving") != ""))

More problematic calories per serving fields: `8 out of 4570 values: ["3291 kcal/kg; 29 kcal/treat", "3200 kcal/kg; 18 kcal/treat", … "varied"]`.

In [224]:
tjhp.shape

(4394, 13)

In [225]:
### DROP THEM. come back later to salvage whichever.
tjhp = tjhp.filter(tjhp['calories_per_serving'].str.contains('^\\s*\\d+\\s*$'))
tjhp.shape
# Yes, this filters out those 8 values giving us trouble before

(4386, 13)

In [226]:
tjhp = tjhp.with_columns(
    pl.col('calories_per_serving').str.strip_chars(' ').cast(pl.Float32).alias('calories_per_serving')
)

Now same for servings per container....

In [227]:
tjhp['nutrition'][0][0]['servings_per_container']

'Serves 1'

In [230]:
# Create raw servings column
tjhp = tjhp.with_columns(
    pl.col("nutrition").map_elements(lambda x: x[0]['servings_per_container'] if len(x)>0 else None, return_dtype=str).alias("servings_per_container_text")
)

In [232]:
unique_spc = tjhp['servings_per_container'].unique() # alue_counts()['servings_per_container']
for spc in unique_spc:
    print(spc)

Serves About 6.5
about 5
Serves 50
Serves About 7
about 4
Serves 57
Servings Varied
Serves About 4.5
Serves about 60
Serves About 3
Serves 12 per box
serves about 22
Serves ABOUT 3
Serves About 25
 serves 1
Serves 74
Serves about 1.5
about 17
Serves about 20
Serves approx. 20
Serves 14
Serves 90
Serves about 28
Serves About 5 servings per container
Serves about 28-30
Serves about 81
Serves about 59
Serves about 384
Serves 567
Serves about 83
Serves 1 bottle
Serves 19
Serves 85
Serves about 40
Serves 7
Serves 500
Serves about 18
Serves 40
Serves about 542
Serves varied
Serves 8 on 8oz and 12 on 12oz
Serves about 47
Serves about 107
Serves 465
Serves about 3
Serves 68
Serves 39
Serves 88
Serves 5.67
Serves about 3.5
Serves about 62
Serves About 4
Serves about 2.5
Serves 120
Serves does not need this since there is only one serving
Serves about 74
Serves 70
Serves Varied
Serves about 13
Serves 5.5
Serves about 68
Serves about 45
Serves about 102
Serves 48
Serves about 4
Serves about 10
Se

In [233]:
tjhp.shape

(4386, 15)

In [234]:
# tjhp.head(3)

The common problems with servings per container include:
* `Serves one` and `Serves does not need this since there is only one serving` (manually replace with 1)
* `Serves about 3 (About 2.5 without Dressing)` (just grab first numeric)
* `Serves 8 on 8oz and 12 on 12oz` (This might be a problem depending on whether there is a single reported calorie value)
* `Serves varied` (idek man)

In [235]:

tjhp = tjhp.with_columns(
    servings_per_container=pl.when(pl.col("servings_per_container").str.contains(' one', literal=True))
    .then(1)
    .otherwise(pl.col("servings_per_container"))
)#.filter(tjhp['servings_per_container'].str.contains(' one', literal=True))

In [236]:
# (?i)(serves about)|(serves)|(about)
tjhp = tjhp.with_columns(
    servings_per_container=tjhp['servings_per_container'].str.extract(r'(\d+)').cast(pl.Float32)
)

In [237]:
tjhp = tjhp.with_columns(
    calories_per_container = tjhp['calories_per_serving'] * tjhp['servings_per_container']
)

In [238]:
tjhp = tjhp.with_columns(
    dollars_per_calorie = tjhp['price_usd'] / tjhp['calories_per_container']
)

In [239]:
tjhp = tjhp.with_columns(
    calories_per_dollar = 1 / pl.col('dollars_per_calorie')
)

In [240]:
tjhp.columns

['sku',
 'name',
 'availability',
 'stock_status',
 'only_x_left_in_stock',
 'nutrition',
 'ingredients',
 'popularity',
 'price',
 'country_of_manufacture',
 'country_of_origin',
 'price_usd',
 'calories_per_serving',
 'servings_per_container',
 'servings_per_container_text',
 'calories_per_container',
 'dollars_per_calorie',
 'calories_per_dollar']

In [241]:
# Remove 0-calorie nothings like salt and tea and hot sauce
# Remove items that are just fucking free i guess??!!
tjhp_trim = tjhp[:, ['sku', 'name', 'calories_per_dollar', 'dollars_per_calorie', 'price_usd', 'calories_per_container', 'calories_per_serving', 'servings_per_container', ]].filter(
    ~tjhp['dollars_per_calorie'].is_null() & ~tjhp['dollars_per_calorie'].is_nan() & (tjhp['calories_per_container']!=0) & (tjhp['price_usd']!=0)
)
tjhp_trim

sku,name,calories_per_dollar,dollars_per_calorie,price_usd,calories_per_container,calories_per_serving,servings_per_container
str,str,f64,f64,f64,f32,f32,f32
"""075259""","""MEAL CASHEW CURRY & RICE""",152.882206,0.006541,3.99,610.0,610.0,1.0
"""076838""","""HOL UNEXPECTED CHEDDAR SPREAD-…",162.324649,0.00616,4.99,810.0,90.0,9.0
"""066838""","""SALMON HOT SMOKED 5 OZ""",36.72788,0.027227,5.99,220.0,110.0,2.0
"""054840""","""KALE & CAULIFLOWER CURRY SALAD""",75.18797,0.0133,3.99,300.0,150.0,2.0
"""074671""","""TUNA SOLID WHITE ALBACORE SALT…",65.326633,0.015308,1.99,130.0,130.0,1.0
"""054114""","""MRNTD GRLD ARTICHOKE HALVES""",100.286533,0.009971,3.49,350.0,50.0,7.0
"""035881""","""BREADED COD FILLETS""",93.437152,0.010702,8.99,840.0,280.0,3.0
"""008990""","""SUPERSWEET CUT WHITE CORN""",268.456376,0.003725,1.49,400.0,80.0,5.0
"""082847""","""EGGS LARGE BROWN DOZEN""",89.552239,0.011167,4.69,420.0,35.0,12.0


Lol why are there polars `null` and also just `NaN` brother

In [242]:
tjhp_trim.sort(by='dollars_per_calorie', descending=False)
# Chocolatey dipping kit limited time only, caloric maximum is a fucking cryptid, it's lost media
# NO these fucking prices are just straight up wrong, 1 cent???

sku,name,calories_per_dollar,dollars_per_calorie,price_usd,calories_per_container,calories_per_serving,servings_per_container
str,str,f64,f64,f64,f32,f32,f32
"""077826""","""KIT CHOCOLATE DIPPING""",120000.0,0.000008,0.01,1200.0,150.0,8.0
"""073468""","""INCREDISAUCE""",100000.0,0.00001,0.01,1000.0,100.0,10.0
"""077328""","""CINNAMON SWIZZLE STICKS""",72000.0,0.000014,0.01,720.0,120.0,6.0
"""077036""","""CINNAMON BUN TRUFFLES""",48000.0,0.000021,0.01,480.0,160.0,3.0
"""067699""","""ALMONDS DARK CHOC AMPED-UP""",38000.0,0.000026,0.01,380.0,380.0,1.0
"""092925""","""CRYSTALLIZED CANDIED GINGER""",15969.849246,0.000063,1.99,31780.0,140.0,227.0
"""058600""","""UNCRYSTALLIZED CANDIED GINGER""",15969.849246,0.000063,1.99,31780.0,140.0,227.0
"""077616""","""TJ'S CRUNCHY PEANUT BUTTER ENE…",2722.689076,0.000367,1.19,3240.0,270.0,12.0
"""077618""","""TJ'S CHOCOLATE CHIP ENERGY""",2621.848739,0.000381,1.19,3120.0,260.0,12.0


In [244]:
cfg = pl.Config()
cfg.set_tbl_rows(10)
# with pl.Config(tbl_rows=10):
#     tjhp_trim.sort(by='dollars_per_calorie', descending=False)
tjhp_trim.sort(by='calories_per_dollar', descending=True).head(20)
# How can I do a HAVING filter after the sort? To remove crazy low dollars_per_calorie values
## What did I mean by this. Maybe HAVING after the calories_per_container thing?

sku,name,calories_per_dollar,dollars_per_calorie,price_usd,calories_per_container,calories_per_serving,servings_per_container
str,str,f64,f64,f64,f32,f32,f32
"""077826""","""KIT CHOCOLATE DIPPING""",120000.0,0.000008,0.01,1200.0,150.0,8.0
"""073468""","""INCREDISAUCE""",100000.0,0.00001,0.01,1000.0,100.0,10.0
"""077328""","""CINNAMON SWIZZLE STICKS""",72000.0,0.000014,0.01,720.0,120.0,6.0
"""077036""","""CINNAMON BUN TRUFFLES""",48000.0,0.000021,0.01,480.0,160.0,3.0
"""067699""","""ALMONDS DARK CHOC AMPED-UP""",38000.0,0.000026,0.01,380.0,380.0,1.0
…,…,…,…,…,…,…,…
"""079176""","""HOL FESTIVELY SHAPED PRETZELS""",1333.333333,0.00075,0.99,1320.0,120.0,11.0
"""070185""","""RICE CALROSE""",1285.140562,0.000778,2.49,3200.0,160.0,20.0
"""038985""","""SALTED TORTILLA CHIPS 2 LB""",1283.667622,0.000779,3.49,4480.0,140.0,32.0


Let's filter out products that obviously make no sense, e.g. excessive calories in a container and $0.01 price tags.

In [247]:
tjhp_clean = tjhp_trim.clone() # Deep copy

In [248]:
tjhp_clean = tjhp_clean.sort(by='calories_per_dollar', descending=True)


In [249]:
tjhp_clean['price_usd'].hist()

breakpoint,category,count
f64,cat,u32
3.193,"""[0.01, 3.193]""",1351
6.376,"""(3.193, 6.376]""",2154
9.559,"""(6.376, 9.559]""",266
12.742,"""(9.559, 12.742]""",61
15.925,"""(12.742, 15.925]""",17
19.108,"""(15.925, 19.108]""",6
22.291,"""(19.108, 22.291]""",6
25.474,"""(22.291, 25.474]""",2
28.657,"""(25.474, 28.657]""",0


In [258]:
tjhp_clean.filter(tjhp_clean['price_usd'] <= 0.49)

sku,name,calories_per_dollar,dollars_per_calorie,price_usd,calories_per_container,calories_per_serving,servings_per_container
str,str,f64,f64,f64,f32,f32,f32
"""077826""","""KIT CHOCOLATE DIPPING""",120000.0,0.000008,0.01,1200.0,150.0,8.0
"""073468""","""INCREDISAUCE""",100000.0,0.00001,0.01,1000.0,100.0,10.0
"""077328""","""CINNAMON SWIZZLE STICKS""",72000.0,0.000014,0.01,720.0,120.0,6.0
"""077036""","""CINNAMON BUN TRUFFLES""",48000.0,0.000021,0.01,480.0,160.0,3.0
"""067699""","""ALMONDS DARK CHOC AMPED-UP""",38000.0,0.000026,0.01,380.0,380.0,1.0
…,…,…,…,…,…,…,…
"""042974""","""ORG APPLE STRAWBERRY FRUIT WRA…",91.836735,0.010889,0.49,45.0,45.0,1.0
"""042973""","""ORG APPLE RASPBERRY FRUIT WRAP""",91.836735,0.010889,0.49,45.0,45.0,1.0
"""081316""","""ORG SOUR WATERMELON BAR""",91.836735,0.010889,0.49,45.0,45.0,1.0


In [259]:
# tjhp = tjhp.with_columns(
#     calories_per_dollar = 1 / pl.col('dollars_per_calorie')
# )

# df.filter REMOVES (filters lol) things that satisfy the condition
tjhp_clean = tjhp_clean.filter(
        (tjhp_clean['calories_per_container'] < 30000) # Demon core candied ginger
        &
        (tjhp_clean['price_usd'] >= 0.48) # Threshold to just include fruit wraps and "just a handful"-type products
)

In [260]:
# tjhp.filter(~tjhp['servings_per_container'].str.contains('[(serves)(Serves)(about)]'))

In [261]:
# # Energy Bar Peanut Butter / tjhp'S CRUNCHY PEANUT BUTTER ENERGY
# # Wrong, it is listing price of a single bar, but servings as 12 bars!!!!
# tjhp.filter(tjhp['sku'] == '077616')

# # Same issue with tjhp'S CHOCOLATE CHIP ENERGY

In [262]:
tjhp.filter(tjhp['sku'] == '070379')

sku,name,availability,stock_status,only_x_left_in_stock,nutrition,ingredients,popularity,price,country_of_manufacture,country_of_origin,price_usd,calories_per_serving,servings_per_container,servings_per_container_text,calories_per_container,dollars_per_calorie,calories_per_dollar
str,str,str,str,null,list[struct[7]],list[struct[2]],str,struct[1],null,str,f64,f32,f32,str,f32,f64,f64
"""070379""","""OIL CANOLA SPRAY ORGANIC""","""1""","""OUT_OF_STOCK""",null,"[{""0 "",[{""0 g"",1,""Total Fat"",""0""}, {""0 mg"",2,""Sodium"",""0""}, … {""0 g"",10,""Protein"",""""}],0,1,"""",""1/3 second spray (0.25g)"",""Serves 536""}]","[{1,""ORGANIC CANOLA OIL""}]","""52""","{{{2.99,""USD""}}}",null,"""Product of Other (do not use) …",2.99,0.0,536.0,"""Serves 536""",0.0,inf,0.0


In [263]:
tjhp.filter(tjhp['sku'] == '073468')

sku,name,availability,stock_status,only_x_left_in_stock,nutrition,ingredients,popularity,price,country_of_manufacture,country_of_origin,price_usd,calories_per_serving,servings_per_container,servings_per_container_text,calories_per_container,dollars_per_calorie,calories_per_dollar
str,str,str,str,null,list[struct[7]],list[struct[2]],str,struct[1],null,str,f64,f32,f32,str,f32,f64,f64
"""073468""","""INCREDISAUCE""","""1""","""OUT_OF_STOCK""",null,"[{""100 "",[{""7 g"",1,""Total Fat"","".09""}, {""1 g"",2,""Saturated Fat"","".05""}, … {""20 mg"",14,""Potassium"",""0""}],0,1,"""",""2 Tbsp. (32g)"",""Serves about 10""}]","[{1,""WATER""}, {2,""CANE SUGAR""}, … {20,""PAPRIKA OLEORESIN FOR COLOR.""}]","""29""","{{{0.01,""USD""}}}",null,"""Product of United States""",0.01,100.0,10.0,"""Serves about 10""",1000.0,0.00001,100000.0


To sanity check items that do not have accurate servings per container given the price, we can insert another variable for the price per serving to see if that is weird.

In [265]:
tjhp_clean = tjhp_clean.with_columns(
    dollars_per_serving = tjhp_clean['price_usd'] / tjhp_clean['servings_per_container']
)

In [266]:
tjhp_clean.head(3)

sku,name,calories_per_dollar,dollars_per_calorie,price_usd,calories_per_container,calories_per_serving,servings_per_container,dollars_per_serving
str,str,f64,f64,f64,f32,f32,f32,f64
"""077616""","""TJ'S CRUNCHY PEANUT BUTTER ENE…",2722.689076,0.000367,1.19,3240.0,270.0,12.0,0.099167
"""077618""","""TJ'S CHOCOLATE CHIP ENERGY""",2621.848739,0.000381,1.19,3120.0,260.0,12.0,0.099167
"""052029""","""OIL 100% CANOLA""",2303.724928,0.000434,3.49,8040.0,120.0,67.0,0.05209


Also, let's fix some obvious mistakes manually.

In [267]:
tjhp_clean = tjhp_clean.with_columns(
    pl.when(tjhp_clean['sku'].is_in(["077616", "077618"]))
    .then(1)
    .otherwise(pl.col("servings_per_container"))
    .alias("servings_per_container")
)


In [268]:
tjhp_clean = tjhp_clean.with_columns(
    calories_per_container = tjhp_clean['calories_per_serving'] * tjhp_clean['servings_per_container']
)

In [269]:
tjhp_clean = tjhp_clean.with_columns(
    calories_per_dollar = tjhp_clean['calories_per_container'] / tjhp_clean['price_usd']
)

In [270]:
tjhp_clean = tjhp_clean.with_columns(
    dollars_per_calorie = 1 / tjhp_clean['calories_per_dollar']
)

In [271]:
tjhp_clean = tjhp_clean.sort(by='calories_per_dollar', descending=True)

In [336]:
# tjhp['nutrition'][0][0]['details']

In [ ]:

[i['amount'] for i in tjhp['nutrition'][0][0]['details'] if i['nutritional_item']=='Protein']

['11 g']

In [354]:
df = pl.DataFrame({"nutritional_info": tjhp['nutrition']})

# Explode the list of dictionaries column
df = df.explode("nutritional_info")

In [359]:
# tjhp['nutrition'][0][0]

In [370]:
data = [
    [{'amount': '38 g', 'display_seq': 1, 'nutritional_item': 'Total Fat', 'percent_dv': '.49'}, 
     {'amount': '25 g', 'display_seq': 2, 'nutritional_item': 'Saturated Fat', 'percent_dv': '1.25'}, 
     {'amount': '0 g', 'display_seq': 3, 'nutritional_item': 'Trans Fat', 'percent_dv': ''}],
    [{'amount': '15 g', 'display_seq': 4, 'nutritional_item': 'Protein', 'percent_dv': ''},
     {'amount': '10 g', 'display_seq': 5, 'nutritional_item': 'Fiber', 'percent_dv': ''}],
     [{'amount': '38 g', 'display_seq': 1, 'nutritional_item': 'Total Fat', 'percent_dv': '.49'}, 
     {'amount': '25 g', 'display_seq': 2, 'nutritional_item': 'Saturated Fat', 'percent_dv': '1.25'}, 
     {'amount': '0 g', 'display_seq': 3, 'nutritional_item': 'Trans Fat', 'percent_dv': ''}],
    
    [{'amount': '15 g', 'display_seq': 4, 'nutritional_item': 'Protein', 'percent_dv': ''},
     {'amount': '10 g', 'display_seq': 5, 'nutritional_item': 'Fiber', 'percent_dv': ''}]
]

# Create a Polars DataFrame where each inner list is a column
# This is a different structure than the previous example
df = pl.DataFrame({"nutritional_info": data})


In [371]:
df

nutritional_info
list[struct[4]]
"[{""38 g"",1,""Total Fat"","".49""}, {""25 g"",2,""Saturated Fat"",""1.25""}, {""0 g"",3,""Trans Fat"",""""}]"
"[{""15 g"",4,""Protein"",""""}, {""10 g"",5,""Fiber"",""""}]"
"[{""38 g"",1,""Total Fat"","".49""}, {""25 g"",2,""Saturated Fat"",""1.25""}, {""0 g"",3,""Trans Fat"",""""}]"
"[{""15 g"",4,""Protein"",""""}, {""10 g"",5,""Fiber"",""""}]"


In [ ]:

# Explode the list of dictionaries column
df = df.explode("nutritional_info")

# Extract the 'amount' where 'nutritional_item' is 'Protein' from the exploded data
protein_amount = df.filter(pl.col("nutritional_info").struct.field("nutritional_item") == "Protein").select(pl.col("nutritional_info").struct.field("amount"))

print(protein_amount)

In [389]:
tjhp['nutrition'][0][0]

{'calories_per_serving': '610 ',
 'details': [{'amount': '38 g',
   'display_seq': 1,
   'nutritional_item': 'Total Fat',
   'percent_dv': '.49'},
  {'amount': '25 g',
   'display_seq': 2,
   'nutritional_item': 'Saturated Fat',
   'percent_dv': '1.25'},
  {'amount': '0 g',
   'display_seq': 3,
   'nutritional_item': 'Trans Fat',
   'percent_dv': ''},
  {'amount': '0 mg',
   'display_seq': 4,
   'nutritional_item': 'Cholesterol',
   'percent_dv': '0'},
  {'amount': '500 mg',
   'display_seq': 5,
   'nutritional_item': 'Sodium',
   'percent_dv': '.22'},
  {'amount': '58 g',
   'display_seq': 6,
   'nutritional_item': 'Total Carbohydrate',
   'percent_dv': '.21'},
  {'amount': '4 g',
   'display_seq': 7,
   'nutritional_item': 'Dietary Fiber',
   'percent_dv': '.14'},
  {'amount': '4 g',
   'display_seq': 8,
   'nutritional_item': 'Total Sugars',
   'percent_dv': ''},
  {'amount': '0 g Added Sugars',
   'display_seq': 9,
   'nutritional_item': 'Includes',
   'percent_dv': '0'},
  {'amoun

In [ ]:
# Try to extract nutrients (not calories)
# Start with protein 
tjhp_p = tjhp.with_columns(
    # tj['nutrition']
    pl.col("nutrition").map_elements(lambda x: x[0]['details'] if len(x)>0 else None).alias("nuts")
    # pl.col("nutrition").map_elements(lambda x: x[0]['servings_per_container'] if len(x)>0 else None, return_dtype=str).alias("servings_per_container")

)
# tjhp.head(3)


C:\Users\riley\AppData\Local\Temp\ipykernel_22168\2695420405.py:3: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tjhp_p = tjhp.with_columns(


In [419]:
tjhp_p = tjhp_p.with_columns(
    pl.col('nuts').map_elements(lambda x: [i['amount'] for i in x if i['nutritional_item']=='Protein'], return_dtype=list[str]).alias('protein')
)


Protein per serving, not just protein.

In [421]:
x = None

In [ ]:
def parse_nutritional_item(x, nutritional_unit=' g'):
    '''Assumes x is a list of one element'''
    try:
        return x[0].strip(nutritional_unit)
    except TypeError:
        return 0

tjhp_p.with_columns(
    pl.col("protein").map_elements(lambda x: parse_nutritional_item(x), return_dtype=float)
)

Save.

In [ ]:
# tjhp_clean.write_csv("data/tjhp_clean.csv")

#### Load Hyde Park.

In [ ]:
tjhp_clean = pl.read_csv("data/tjhp_clean.csv")

### Hyde Park Viz

In [312]:
import altair as alt

# alt.Chart(tjhp_clean).mark_point().encode(
#     x='price_usd',
#     y='calories_per_container',
#     color='calories_per_serving',
#     tooltip=['sku', 'name']
# ).interactive()

# https://github.com/vega/altair/issues/1245
# How to disable automatic alphabetic sorting of x value
alt.Chart(tjhp_clean.sort(by='dollars_per_calorie', descending=False).head(20)).mark_bar().encode(
    # x='name',
    x=alt.X('name', title='Product', type='nominal', sort=None, axis=alt.Axis(labelAngle=-45)),
    y=alt.Y(field='dollars_per_100calorie', type='quantitative', title='$ per 100 Calories'),
    color=alt.Color('dollars_per_serving', title='$ per Serving').scale(scheme="greens"),
    tooltip=['sku', 'name', 'price_usd', 'dollars_per_calorie', 'servings_per_container']
).transform_calculate(
    dollars_per_100calorie='datum.dollars_per_calorie * 100' # Calculate the new field
).interactive(

).properties(
    title='Hyde Park TJ\'s 20 items with the cheapest calories',
    width=800,
    height=300
).configure_title(
    fontSize=24,
    # font='IMPACT',
    anchor='middle',
    color='black'
).configure_axis(
    labelFontSize=15,
    titleFontSize=20
)

# tjhp_clean.columns

alt.Chart(...)

In [315]:
pretzel = [i for i in all_items_hp if i['sku']=='079176']

In [316]:
len(pretzel)

1

In [329]:
tjhp['nutrition'][0][0]['details']

[{'amount': '38 g',
  'display_seq': 1,
  'nutritional_item': 'Total Fat',
  'percent_dv': '.49'},
 {'amount': '25 g',
  'display_seq': 2,
  'nutritional_item': 'Saturated Fat',
  'percent_dv': '1.25'},
 {'amount': '0 g',
  'display_seq': 3,
  'nutritional_item': 'Trans Fat',
  'percent_dv': ''},
 {'amount': '0 mg',
  'display_seq': 4,
  'nutritional_item': 'Cholesterol',
  'percent_dv': '0'},
 {'amount': '500 mg',
  'display_seq': 5,
  'nutritional_item': 'Sodium',
  'percent_dv': '.22'},
 {'amount': '58 g',
  'display_seq': 6,
  'nutritional_item': 'Total Carbohydrate',
  'percent_dv': '.21'},
 {'amount': '4 g',
  'display_seq': 7,
  'nutritional_item': 'Dietary Fiber',
  'percent_dv': '.14'},
 {'amount': '4 g',
  'display_seq': 8,
  'nutritional_item': 'Total Sugars',
  'percent_dv': ''},
 {'amount': '0 g Added Sugars',
  'display_seq': 9,
  'nutritional_item': 'Includes',
  'percent_dv': '0'},
 {'amount': '11 g',
  'display_seq': 10,
  'nutritional_item': 'Protein',
  'percent_dv':

### All list

In [ ]:
import polars as pl

In [ ]:
# pl.json_normalize(all_items_fixed[100:102])

Beyond just dumping each json object from `all_items_fixed` into `pl.json_normalize()`, we also need to further handle the `nutrition` and `ingredients` dictionaries.

In [ ]:
pl.json_normalize(all_items_fixed[100]['nutrition'])

For now, just save it as expressly as possible.

In [ ]:
# df_all_items_raw = pl.json_normalize(all_items_fixed,
#                                      infer_schema_length=None)

In [ ]:
df_all_items_raw.tail(3)
# Columns 2 and 3 are Lists of Structs of...


In [ ]:
# df_all_items_raw.write_csv('data/all_items_raw.csv')
# # Nested columns 2 and 3 do not work with CSV

In [ ]:
# import json
# all_items_fixed_raw_path = "data/all_items_fixed_raw.json"
# with open(all_items_fixed_raw_path, "w") as f:
#        json.dump(all_items_fixed, f, indent=4)



Every description is empty html???

In [ ]:
broc = [item['name'] for item in all_items_fixed if 'KALE' in item['name']]



broc

In [ ]:
df_all_items_raw.write_csv()

Let's try and isolate only food items.

In [ ]:
pl.json_normalize(all_items_fixed[100], max_level=4)

In [ ]:
nuts = [item['nutrition'] for item in all_items_fixed]

In [ ]:
# Get only items with non-empty nutrition information
# This will be criterion for what counts as "food"

In [ ]:
# nut_lens = [len(nut) for nut in nuts]

Oh, sku 10032021 "POP UP SPONGES" for some reason contains information for pizza crusts? The problematic `"item_description": "Broccoli & Kale Pizza Crust description"`.

#### Getting only the values we need from the json, making it relational

In [ ]:
import json
import polars as pl

In [ ]:
with open('data/all_items_fixed_raw.json', 'r') as f:
    tj_raw = json.load(f)

Recall that the `item_description` field is `null` for just about every entry except one where it is instead the unrelated `Broccoli & Kale Pizza Crust description`.

In [ ]:
tj_raw = [item for item in tj_raw if item['item_description'] != 'Broccoli & Kale Pizza Crust description']

In [ ]:
tj = pl.DataFrame(tj_raw)

Need to:

* Remove rows with null `nutrition`
* Remove useless fields (`item_description`, `description`)
* Turn nested fields into 
    * lists (ingredients)
    * individual and possibly sparse columns (fiber, protein, calories)

In [ ]:
tj = tj.drop(['item_description', 'description'])

In [ ]:
tj.shape

In [ ]:
tj = tj.drop_nulls(subset=['nutrition', 'price'])
tj.shape

In [ ]:
tj.head()

In [ ]:
tj['price'].str

In [ ]:
temp = tj.with_columns(
    pl.col("price").struct.field("regularPrice").struct.field("amount").struct.field("currency").alias("nested_field")
)

print(temp)

In [ ]:
temp['nested_field'].value_counts()

OK, they are all in USD, proceed.

In [ ]:
tj = tj.with_columns(
    pl.col("price").struct.field("regularPrice").struct.field("amount").struct.field("value").alias("price_usd")
)


Onto the same nested json but for calories...

Wow, these nutrition items are formatted horrendously. The key is not a key whatsoever. Should go back and fix this in the original GraphQL query under `nutrition`.

In [ ]:
tj['nutrition'][0]

Oh, maybe not.

In [ ]:
tj['nutrition'][0][0]

In [ ]:
tj['nutrition'][0][0]['calories_per_serving']

In [ ]:
tj = tj.with_columns(
    # tj['nutrition']
    pl.col("nutrition").map_elements(lambda x: x[0]['calories_per_serving'] if len(x)>0 else None, return_dtype=str).alias("calories_per_serving")
)
tj.head(3)

These things definitely do not actually have 0 calories. Drop them.

In [ ]:
tj.filter((pl.col("calories_per_serving") == ""))

In [ ]:
tj = tj.filter((pl.col("calories_per_serving") != ""))

More problematic calories per serving fields: `8 out of 4570 values: ["3291 kcal/kg; 29 kcal/treat", "3200 kcal/kg; 18 kcal/treat", … "varied"]`.

In [ ]:
tj.shape

In [ ]:
### DROP THEM. come back later to salvage whichever.
tj = tj.filter(tj['calories_per_serving'].str.contains('^\\s*\\d+\\s*$'))
tj.shape
# Yes, this filters out those 8 values giving us trouble before

In [ ]:
tj = tj.with_columns(
    pl.col('calories_per_serving').str.strip_chars(' ').cast(pl.Float32).alias('calories_per_serving')
)

Now same for servings per container....

In [ ]:
tj['nutrition'][0][0]['servings_per_container']

In [ ]:
# Create raw servings column
tj = tj.with_columns(
    pl.col("nutrition").map_elements(lambda x: x[0]['servings_per_container'] if len(x)>0 else None, return_dtype=str).alias("servings_per_container")
)

In [ ]:
unique_spc = tj['servings_per_container'].unique() # alue_counts()['servings_per_container']
for spc in unique_spc:
    print(spc)

In [ ]:
tj.shape

In [ ]:
# tj.head(3)

The common problems with servings per container include:
* `Serves one` and `Serves does not need this since there is only one serving` (manually replace with 1)
* `Serves about 3 (About 2.5 without Dressing)` (just grab first numeric)
* `Serves 8 on 8oz and 12 on 12oz` (This might be a problem depending on whether there is a single reported calorie value)
* `Serves varied` (idek man)

In [ ]:

tj = tj.with_columns(
    spc=pl.when(pl.col("servings_per_container").str.contains(' one', literal=True))
    .then(1)
    .otherwise(pl.col("servings_per_container"))
)#.filter(tj['servings_per_container'].str.contains(' one', literal=True))

In [ ]:
# (?i)(serves about)|(serves)|(about)
tj = tj.with_columns(
    spc=tj['spc'].str.extract(r'(\d+)').cast(pl.Float32)
)

In [ ]:
tj = tj.with_columns(
    calories_per_container = tj['calories_per_serving'] * tj['spc']
)

In [ ]:
tj = tj.with_columns(
    dollars_per_calorie = tj['price_usd'] / tj['calories_per_container']
)

In [ ]:
tj = tj.with_columns(
    calories_per_dollar = 1 / pl.col('dollars_per_calorie')
)

In [ ]:
tj.columns

In [ ]:
# Remove 0-calorie nothings like salt and tea and hot sauce
# Remove items that are just fucking free i guess??!!
tj_trim = tj[:, ['sku', 'name', 'calories_per_dollar', 'dollars_per_calorie', 'price_usd', 'calories_per_container', 'calories_per_serving', 'spc', ]].filter(
    ~tj['dollars_per_calorie'].is_null() & ~tj['dollars_per_calorie'].is_nan() & (tj['calories_per_container']!=0) & (tj['price_usd']!=0)
)
tj_trim

Lol why are there polars `null` and also just `NaN` brother

In [ ]:
tj_trim.sort(by='dollars_per_calorie', descending=False)
# Chocolatey dipping kit limited time only, caloric maximum is a fucking cryptid, it's lost media
# NO these fucking prices are just straight up wrong, 1 cent???

In [ ]:
cfg = pl.Config()
cfg.set_tbl_rows(20)
# with pl.Config(tbl_rows=10):
#     tj_trim.sort(by='dollars_per_calorie', descending=False)
tj_trim.sort(by='calories_per_dollar', descending=True).head(20)
# How can I do a HAVING filter after the sort? To remove crazy low dollars_per_calorie values
## What did I mean by this. Maybe HAVING after the calories_per_container thing?

In [ ]:
# Energy Bar Peanut Butter / TJ'S CRUNCHY PEANUT BUTTER ENERGY
# Wrong, it is listing price of a single bar, but servings as 12 bars!!!!
tj.filter(tj['sku'] == '077616')

# Same issue with TJ'S CHOCOLATE CHIP ENERGY

In [ ]:
tj.filter(tj['sku'] == '070379')

In [ ]:
tj.filter(tj['sku'] == '073468')

Let's filter out products that obviously make no sense, e.g. excessive calories in a container and $0.01 price tags.

In [ ]:
tj_clean = tj_trim.clone() # Deep copy

In [ ]:
tj_clean = tj_clean.sort(by='calories_per_dollar', descending=True)


In [ ]:
tj_clean['price_usd'].hist()

In [ ]:
tj_clean.filter(tj_clean['price_usd'] <= 0.49)

In [ ]:
# tj = tj.with_columns(
#     calories_per_dollar = 1 / pl.col('dollars_per_calorie')
# )

# df.filter REMOVES (filters lol) things that satisfy the condition
tj_clean = tj_clean.filter(
        (tj_clean['calories_per_container'] < 30000) # Demon core candied ginger
        &
        (tj_clean['price_usd'] >= 0.49) # Threshold at fruit wraps and "just a handful"-type products
)

In [ ]:
# tj.filter(~tj['servings_per_container'].str.contains('[(serves)(Serves)(about)]'))

In [ ]:
# tj_clean_top20 = 
# tj_clean.sort(by='calories_per_dollar', descending=True).head(20)

To sanity check items that do not have accurate servings per container given the price, we can insert another variable for the price per serving to see if that is weird.

In [ ]:
tj_clean = tj_clean.with_columns(
    dollars_per_serving = tj_clean['price_usd'] / tj_clean['spc']
)

In [ ]:
tj_clean.head(3)

Also, let's fix some obvious mistakes manually.

In [ ]:
tj_clean = tj_clean.with_columns(
    pl.when(tj_clean['sku'].is_in(["077616", "077618"]))
    .then(1)
    .otherwise(pl.col("spc"))
    .alias("spc")
)


In [ ]:
tj_clean = tj_clean.with_columns(
    calories_per_container = tj_clean['calories_per_serving'] * tj_clean['spc']
)

In [ ]:
tj_clean = tj_clean.with_columns(
    calories_per_dollar = tj_clean['calories_per_container'] / tj_clean['price_usd']
)

In [ ]:
tj_clean = tj_clean.with_columns(
    dollars_per_calorie = 1 / tj_clean['calories_per_dollar']
)

In [ ]:
tj_clean = tj_clean.sort(by='calories_per_dollar', descending=True)

Save.

In [ ]:
tj_clean.write_csv("data/tj_clean.csv")

Load.

In [ ]:
tj_clean

## Viz

In [ ]:
import altair as alt

In [ ]:

alt.Chart(tj_clean).mark_point().encode(
    x='price_usd',
    y='calories_per_container',
    color='calories_per_serving',
    tooltip=['sku', 'name']
).interactive()

In [ ]:
# https://github.com/vega/altair/issues/1245
# How to disable automatic alphabetic sorting of x value
alt.Chart(tj_clean.sort(by='calories_per_dollar', descending=True).head(20)).mark_bar().encode(
    # x='name',
    x=alt.X('name', type='nominal', sort=None, axis=alt.Axis(labelAngle=-45)),
    y='calories_per_dollar',
    color=alt.Color('dollars_per_serving').scale(scheme="greens"),
    tooltip=['sku', 'name', 'price_usd', 'calories_per_serving', 'spc']
).interactive(

).properties(
    width=800,
    height=300
)

In [ ]:
tj_clean.columns

## Misc

In [ ]:
import pandas as pd

In [ ]:
ptj = pd.DataFrame(tj)

In [ ]:
ptj.columns = tj.columns

In [ ]:
ptj.head(3)

In [ ]:
ptj['nutrition'][0][0]

In [ ]:
ptj

In [ ]:
# ptj['nutrition'].apply(lambda x: x[0] if len(x)>0 else None)

In [ ]:
# tj['nutrition'].struct.field('calories_per_serving')

In [ ]:
# tj.with_columns(
#     pl.col("nutrition").map_elements(lambda x: x[0][0]).struct.field('calories_per_serving').alias("calories_per_serving")
# )

In [ ]:
# temp = tj.with_columns(
#     pl.col("nutrition").struct.field("calories_per_serving").alias("calories_per_serving")
# )
# temp